In [1]:
import ROOT
import pandas as pd
import numpy as np
import os
import sys
import ipynbname
from pathlib import Path

project_root = str(ipynbname.path().parent.parent)
sys.path.append(project_root)

from processing import SiphraAcquisition, fit_peak_expbg

# ROOT.gROOT.SetBatch(False)
# ROOT.gEnv.SetValue("Canvas.ShowEventStatus", 1)
# ROOT.gEnv.SetValue("Canvas.ShowToolBar", 1)
# ROOT.gEnv.SetValue("Canvas.UseGL", 0)
# ROOT.gROOT.ProcessLine("gVirtualX = new TGX11();")


In [2]:
# Constants
BITS12 = 2**12
BITS9 = 2**9 # 512 typical number of bins used
# Energy emission peaks in MeV
K40_MEV = 1.460
TL208_MEV = 2.614

In [5]:
acq = SiphraAcquisition(project_root+'/data/260209/1_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Background.csv',
                        active_chs=[2,4],
                        exposure_sec=119.634,
                        sipm_chs="5-8")

In [ ]:
dfs_SiPM58_BG = []
dfs_SiPM58_BG.append(pd.read_csv('../data/260209/1_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM58_BG.append(pd.read_csv('../data/260209/2_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM58_BG.append(pd.read_csv('../data/260209/3_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM58_BG.append(pd.read_csv('../data/260209/4_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Background.csv'))
times_SiPM58_BG = [119.634, 161.062, 148.920, 164.828]

dfs_SiPM14_BG = []
dfs_SiPM14_BG.append(pd.read_csv('../data/260209/5_SiPM_ChannelsTest_Ch1-4_Ch2_CT_Trh1_cmisvoffset127_Background.csv'))
dfs_SiPM14_BG.append(pd.read_csv('../data/260209/2_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM14_BG.append(pd.read_csv('../data/260209/3_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Background.csv'))
dfs_SiPM14_BG.append(pd.read_csv('../data/260209/4_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Background.csv'))
times_SiPM14_BG = [34.836, 80.792, 54.386, 90.837]

dfs_SiPM14_Cs137 = []
dfs_SiPM14_Cs137.append(pd.read_csv('../data/260209/12_SiPM_ChannelsTest_Ch1-4_Ch2_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM14_Cs137.append(pd.read_csv('../data/260209/11_SiPM_ChannelsTest_Ch1-4_Ch6_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM14_Cs137.append(pd.read_csv('../data/260209/10_SiPM_ChannelsTest_Ch1-4_Ch10_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM14_Cs137.append(pd.read_csv('../data/260209/9_SiPM_ChannelsTest_Ch1-4_Ch14_CT_Thr1_cmisvoffset127_Cs137.csv'))
times_SiPM14_Cs137 = [23.381, 24.606, 24.144, 24.645]

dfs_SiPM58_Cs137 = []
dfs_SiPM58_Cs137.append(pd.read_csv('../data/260209/13_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM58_Cs137.append(pd.read_csv('../data/260209/14_SiPM_ChannelsTest_Ch5-8_Ch6_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM58_Cs137.append(pd.read_csv('../data/260209/15_SiPM_ChannelsTest_Ch5-8_Ch10_CT_Thr1_cmisvoffset127_Cs137.csv'))
dfs_SiPM58_Cs137.append(pd.read_csv('../data/260209/16_SiPM_ChannelsTest_Ch5-8_Ch14_CT_Thr1_cmisvoffset127_Cs137.csv'))
times_SiPM58_Cs137 = [24.720, 24.916, 24.848, 24.934]

SIPHRA_Ch_list = ['Ch2', 'Ch6', 'Ch10', 'Ch14']
dfs = [dfs_SiPM14_BG, dfs_SiPM14_Cs137, dfs_SiPM58_BG, dfs_SiPM58_Cs137]
data_SiPM_chs = ['1-4', '1-4', '5-8', '5-8']
legend = ['Ch.2', 'Ch.6', 'Ch.10', 'Ch.14']

In [5]:
lst = [_ for _ in range(18)]
lst

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [8]:
all(isinstance(_, int) and 0 <= _ <= 17 for _ in lst)

True

In [9]:
list(lst)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [24]:
from collections import UserDict
import numpy as np
class ChDataDict(UserDict):
    def __setitem__(self, key, value):
        if not isinstance(key, int):
            raise TypeError("Key must be an integer")
        if not isinstance(value, np.ndarray):
            raise TypeError("Value must be a numpy array")
        super().__setitem__(key, value)

chdata = ChDataDict({1: np.array([1, 2, 3]), 3: np.array([4, 5, 6])})
# chdata[1] = np.array([1,2,3])
# chdata[3] = np.array([4,5,6])

chdata[3]
k = list(chdata.keys())
k[0]

1

In [12]:
a = 2
list(a)

TypeError: 'int' object is not iterable

In [17]:
ll = []
l = [1,2,3,4]
ll.append(l)
ll

TypeError: list.append() takes exactly one argument (4 given)

In [5]:
df = pd.read_csv(project_root+'/data/260209/1_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Background.csv')

In [6]:
from pathlib import Path

Path(df)

TypeError: argument should be a str or an os.PathLike object where __fspath__ returns a str, not 'DataFrame'

In [7]:
pp = Path('/data/260209/1_SiPM_ChannelsTest_Ch5-8_Ch2_CT_Thr1_cmisvoffset127_Background.csv').resolve()

In [9]:
pp.suffix

'.csv'